## Problem Statement

Develop an ML model using supervised learning from the proposed dataset. The problem is a binary classification problem and the target variable is 'isFraud'. The model should predict for the positive class, which indicates that a transaction is fraudulent. The model should additionally be written in the Tensorflow framework.

## Exploratory Data Analysis (EDA)

#### Imports

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

#### Set the dataset file path to where you stored the csv file on your system.

In [2]:
dataset_file_path = 'data/financial_transactions.csv'

In [ ]:
# Better fitting column names, removes some of the typos in the original dataset column names
column_names = ['step', 'paymentType', 'amount', 'accSender', 'oldBalanceSender', 'newBalanceSender', 'accRecipient', 'oldBalanceRecipient',
                'newBalanceRecipient', 'isFraud', 'isFlaggedFraud']

df = pd.read_csv(dataset_file_path)
df.head()

/tmp/ipykernel_1227/3436451484.py:5: DtypeWarning: Columns (0,2,4,5,7,8,9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(dataset_file_path, names=column_names)


,step,paymentType,amount,accSender,oldBalanceSender,newBalanceSender,accRecipient,oldBalanceRecipient,newBalanceRecipient,isFraud,isFlaggedFraud
0,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
1,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
2,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
3,1,TRANSFER,181.0,C1305486145,181.0,0.0,C553264065,0.0,0.0,1,0
4,1,CASH_OUT,181.0,C840083671,181.0,0.0,C38997010,21182.0,0.0,1,0


## Data Preprocessing

In [ ]:
from sklearn.model_selection import train_test_split